In [ ]:
import pandas as pd
import os
from google.colab import drive
import keras
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
%cd /content/drive/My Drive/CS 5242 Project/New codes

/content/drive/.shortcut-targets-by-id/1ceyXZ-lMKQQU1-faVxirzfM9tb2pOj3b/CS 5242 Project/New codes


In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
# load model
model = ResNet50()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
def preprocess_and_generate_per_video(video_path):
  # path = os.listdir(video_path)[0]
  # image = load_img(os.path.join(video_path, path), target_size=(224, 224))
  # print(img_to_array(image).shape)
  images = np.zeros(shape=(30, 224, 224, 3))
  i = 0
  for file in sorted(os.listdir(video_path)):
    # load an image from file
    image_path = os.path.join(video_path, file)
    image = load_img(image_path, target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = np.asarray(preprocess_input(image))
    # print(type(image))
    images[i] = image[0]
    i += 1  
  return model.predict(images)


In [ ]:
train_path = 'Data/train'
save_path = 'Data/train_cnn'

step_i = 0
for video in sorted(os.listdir(train_path)):
  print(step_i)
  features = preprocess_and_generate_per_video(os.path.join(train_path, video))
  np.save('{}/train_cnnfeatures{}.npy'.format(save_path, step_i), features)
  step_i += 1

In [ ]:
test_path = 'Data/test'
save_path = 'Data/test_cnn'

step_i = 0
for video in sorted(os.listdir(test_path)):
  print(step_i)
  features = preprocess_and_generate_per_video(os.path.join(test_path, video))
  np.save('{}/test_cnnfeatures{}.npy'.format(save_path, step_i), features)
  step_i += 1

In [ ]:
train_path = 'Data/optical_flow_images/train'
save_path = 'Data/optical_cnn/train_cnn'
step_i = 0
for video in sorted(os.listdir(train_path)):
  print(step_i)
  features = preprocess_and_generate_per_video(os.path.join(train_path, video))
  np.save('{}/train_cnnfeatures{}.npy'.format(save_path, step_i), features)
  step_i += 1

In [ ]:
test_path = 'Data/optical_flow_images/test'
save_path = 'Data/optical_cnn/test_cnn'
step_i = 0
for video in sorted(os.listdir(test_path)):
  print(step_i)
  features = preprocess_and_generate_per_video(os.path.join(test_path, video))
  np.save('{}/test_cnnfeatures{}.npy'.format(save_path, step_i), features)
  step_i += 1

In [ ]:
test_path = 'Data/augmented_images'
save_path = 'Data/augmented_cnn'

step_i = 0
for video in sorted(os.listdir(train_path)):
  print(step_i)
  video_num = video.split('-')[0]
  features = preprocess_and_generate_per_video(os.path.join(train_path, video))
  np.save('{}/train_cnnfeatures{}_{}.npy'.format(save_path, step_i, video_num), features)
  step_i += 1

# Checking CNN Feature for single image and batch

In [ ]:
features = preprocess_and_generate_per_video('Data/train/000000')

In [ ]:
image_path = 'Data/train/000000/000001.jpg'
image = load_img(image_path, target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = np.asarray(preprocess_input(image))
feature_0 = model.predict(image)

In [ ]:
feature_0

array([[1.5731274 , 1.226066  , 0.06406283, ..., 5.235763  , 0.36012346,
        0.        ]], dtype=float32)

In [ ]:
features[0]

array([1.5731236 , 1.2260681 , 0.06406257, ..., 5.235759  , 0.3601237 ,
       0.        ], dtype=float32)